In [2]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pandas as pd
import os
import sqlite3

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
folder_path = "/home/ec2-user/SageMaker/data/FullSetAFI"
data_parse_list = [] # Initialize an empty list to hold the parsed data
# Loop through each file in the folder
for filename in os.listdir(folder_path):

    if filename.endswith(".xml"):
        # Construct the full path to the XML file
        xml_file_path = os.path.join(folder_path, filename)

        # Parse the XML file
        tree = ET.parse(xml_file_path)
        root = tree.getroot()

        contributors = root.findall('.//Contributor/ProductionCompany/NormalizedDisplayForm')
        
        # Extract producers and production companies
        goid = root.find(".//GOID").text #search for the string "GOID" in the XML tree
        film_name = root.find(".//TitleAtt/Title").text #search for the string "Title" in the XML tree
        date_released = root.find(".//NumericDate").text #search for the string "NumericDate" in the XML tree
        publisher_country = root.find(".//PublicationInfo/Publisher/PublisherCountryName") #search for the string "PublisherCountryName" in the XML tree
        distribution_companies = root.findall(".//ContribDescAtt[ContribDesc='Distribution Company']/../..") #search for all the strings "ContribDescAtt" with "ContribDesc" equal to "Distribution Company" in the XML tree


        if publisher_country is not None:
            publisher_country = publisher_country.text #if there is information in the PublisherCountryName, extract the text
        else:
            publisher_country = "NA"


        # Initialize lists to hold the extracted data
        producers_list = []
        production_companies_list = []
        exec_producer_list = []
        organization_name_list = []

        for contributor in root.findall(".//Contributor"):
            for producer in contributor.findall(".//Producer"):
                producer_name = producer.findtext(".//NormalizedDisplayForm") #search for the string "NormalizedDisplayForm" in the XML tree
                producers_list.append(producer_name) #add to the producers list

            for production_company in contributor.findall(".//ProductionCompany"):
                company_name = production_company.findtext(".//NormalizedDisplayForm") #search for the string "NormalizedDisplayForm" in the XML tree
                production_companies_list.append(company_name) #add to the production companies list
 
        for company in distribution_companies: 
            organization_name_element = company.find(".//OrganizationNameAtt/OrganizationName") #search for the string "OrganizationName" in the XML tree
            if organization_name_element is not None: #check if the organization name element exists
                organization_name = organization_name_element.text.strip() #extract the text and remove any leading or trailing whitespace
                organization_name_list.append(organization_name) #add to the organization name list



        # Create a dictionary for each movie
        movie_dictionary = {"Title": film_name, "GOID": goid, "Date": date_released, "Producer": producers_list, 
                            "Production_Companies": production_companies_list, "country": publisher_country, "distribution": organization_name_list}

        # Append the dictionary to the list
        data_parse_list.append(movie_dictionary)
    practice_movies_df = pd.DataFrame(data_parse_list) # Create a DataFrame from the list of dictionaries




In [ ]:
#due to being in a virtual enviroment, I have to save the data as a pickle file that can be export via email as the final output. 
practice_movies_df.to_pickle("./FullSetFII_pickle.pkl")
